In [8]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import json

In [9]:
today = datetime.today().strftime("%Y%m%d")
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [10]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [11]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1


In [12]:
# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links[:20]:    # 숫자 변경
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [48]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer # pip install sentence_transformers
from bareunpy import Tagger # pip install bareunpy

tagger = Tagger('koba-Q2CYNCI-XZ7E7PI-X6YRKPY-K4Z2KMY')
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


In [53]:
def keyword_ext(text):

    tokenized_doc = tagger.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'NNG' or word[1] == 'NNP'])

    n_gram_range = (1,1)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    return mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    # print(keywords_idx)

    return [words[idx] for idx in keywords_idx]


In [55]:
# Create a DataFrame to store titles and nouns
df = pd.DataFrame(columns=["Title", "Keywords"])

# Text analysis using Mecab and store in DataFrame
for i, (title, content) in enumerate(set(news_contents), start=1):
    if content:
        keywords = keyword_ext(content)
        new_row = pd.DataFrame({"Title": [title], "Keywords": [keywords]})
        df = pd.concat([df, new_row], ignore_index=True)        

# Display the DataFrame
print(df)

                                                Title  \
0           "직장 다니는 배우자 월급으로 적자 메워"... 자영업 연체 '역대 최대'   
1                       우리금융, 서울역 '쪽방촌' 주민들 치과 치료 돕는다   
2               국내 조선사 잇따라 선박 수주…한화오션 2.1조·삼성중공업 1.4조   
3   ‘6.7→8.2조’ 석달만에 23% 커진 三電 2Q 예상 영업익…반도체 반등 타고 ...   
4              "입맛 뚝 떨어져" 즉석밥 뜯자 '곰팡이 득실'…제조사 입장 들어보니   
5      “세금이 22%인데 美주식 하는 사람은 멍청이”…어떻게 생각하십니까? [투자360]   
6                     전영현 삼성전자 부회장, 노조 만났다…"노사 대화 노력"   
7       이형주 VM 컨설팅 대표, 한국 복합문화공간의 유니크 베뉴 경쟁력 연구 논문 발표   
8                개인정보위 “알리·테무 조사 결과 이달 발표…하반기도 AI 집중”   
9                        하이로닉, 치과임플란트 신제품 베트남 유통계약 체결   
10      명품감정원이 “진짜” 랬는데…“가짜”라 뒤집은 중고거래플랫폼, 이런 황당한 일이?   

                           Keywords  
0             [금리, 대출, 자금, 소비자, 비용]  
1   [우리금융그룹, 번째, 우리금융미래재단, 서울대, 이날]  
2        [vlcc, 천연가스, 셔틀탱커, 연료, 계약]  
3         [반도체, 삼성전자, 가격, 투자업계, 속도]  
4             [제품, 곰팡이, 무균, 유통, 상품]  
5         [투자자, 주식, 미국, 한국투자증권, 투자]  
6      [노조, 전국삼성전자노동조합, 임금, 협상, 노동]  
7     [컨설팅,